In [3]:
%load_ext autoreload

In [4]:
%autoreload

from instruments.srs.srsdg645 import SRSDG645
from quantities import ns

In [5]:
addr = "192.168.3.1"

In [6]:
inst = SRSDG645.open_tcpip(addr, 5025)

In [35]:
inst.display = (inst.DisplayMode.channel_delay, inst.Channels.C)
inst.trigger_source = inst.TriggerSource.external_rising

inst.channel["D"].delay = (inst.channel["C"], 10*ns)

inst.output["CD"].level_amplitude = 1
inst.output["CD"].level_offset = -1.09
inst.output["CD"].polarity = inst.LevelPolarity.negative

In [12]:
float(inst.channel["C"].delay[1])*1E9

500.0

In [1]:
import subprocess
import xml.etree.ElementTree as ET


def scan_for_hosts(ip_range):
    """Scan the given IP address range using Nmap and return the result
    in XML format.
    """
    nmap_args = ['nmap', '-n', '-sP', '-oX', '-', ip_range]
    return subprocess.check_output(nmap_args)


def find_ip_address_for_mac_address(xml, mac_address):
    """Parse Nmap's XML output, find the host element with the given
    MAC address, and return that host's IP address (or `None` if no
    match was found).
    """
    host_elems = ET.fromstring(xml).iter('host')
    host_elem = find_host_with_mac_address(host_elems, mac_address)
    if host_elem is not None:
        return find_ip_address(host_elem)


def find_host_with_mac_address(host_elems, mac_address):
    """Return the first host element that contains the MAC address."""
    for host_elem in host_elems:
        if host_has_mac_address(host_elem, mac_address):
            return host_elem


def host_has_mac_address(host_elem, wanted_mac_address):
    """Return true if the host has the given MAC address."""
    found_mac_address = find_mac_address(host_elem)
    return (
        found_mac_address is not None and
        found_mac_address.lower() == wanted_mac_address.lower()
    )


def find_mac_address(host_elem):
    """Return the host's MAC address."""
    return find_address_of_type(host_elem, 'mac')


def find_ip_address(host_elem):
    """Return the host's IP address."""
    return find_address_of_type(host_elem, 'ipv4')


def find_address_of_type(host_elem, type_):
    """Return the host's address of the given type, or `None` if there
    is no address element of that type.
    """
    address_elem = host_elem.find('./address[@addrtype="{}"]'.format(type_))
    if address_elem is not None:
        return address_elem.get('addr')


if __name__ == '__main__':
    mac_address = '00:33:66:99:cc:ff'
    ip_range = '192.168.1.1-255'

    xml = scan_for_hosts(ip_range)
    ip_address = find_ip_address_for_mac_address(xml, mac_address)

    if ip_address:
        print('Found IP address {} for MAC address {} in IP address range {}.'
              .format(ip_address, mac_address, ip_range))
    else:
        print('No IP address found for MAC address {} in IP address range {}.'
              .format(mac_address, ip_range))import subprocess
import xml.etree.ElementTree as ET

def find_ip_address_for_mac_address(xml, mac_address):
    """Parse Nmap's XML output, find the host element with the given
    MAC address, and return that host's IP address (or `None` if no
    match was found).
    """
    host_elems = ET.fromstring(xml).iter('host')
    host_elem = find_host_with_mac_address(host_elems, mac_address)
    if host_elem is not None:
        return find_ip_address(host_elem)

def find_ip_address(host_elem):
    """Return the host's IP address."""
    return find_address_of_type(host_elem, 'ipv4')

b'  131.243.163.255       ff-ff-ff-ff-ff-ff     static    \r\n  255.255.255.255       ff-ff-ff-ff-ff-ff     static    \r\n  192.168.255.255       ff-ff-ff-ff-ff-ff     static    \r\n  255.255.255.255       ff-ff-ff-ff-ff-ff     static    \r\n  192.168.255.255       ff-ff-ff-ff-ff-ff     static    \r\n  255.255.255.255       ff-ff-ff-ff-ff-ff     static    \r\n  192.168.255.255       ff-ff-ff-ff-ff-ff     static    \r\n  255.255.255.255       ff-ff-ff-ff-ff-ff     static    \r\n'
131.243.163.255
